In [1]:
import pandas as pd

In [6]:
df=pd.read_csv("./100_Unique_QA_Dataset.csv")
df.head()

question      answer
0                   What is the capital of France?       Paris
1                  What is the capital of Germany?      Berlin
2               Who wrote 'To Kill a Mockingbird'?  Harper-Lee
3  What is the largest planet in our solar system?     Jupiter
4   What is the boiling point of water in Celsius?         100

In [9]:
#tokenize
def tokenize(text):
    text=text.lower()
    text=text.replace('?','')
    text=text.replace("'","")
    return text.split()

In [10]:
tokenize('What is the capital of France?')

['what', 'is', 'the', 'capital', 'of', 'france']

In [11]:
# vocab
vocab = {'<UNK>':0}

In [12]:
def build_vocab(row):
  tokenized_question = tokenize(row['question'])
  tokenized_answer = tokenize(row['answer'])

  merged_tokens = tokenized_question + tokenized_answer

  for token in merged_tokens:

    if token not in vocab:
      vocab[token] = len(vocab)


In [14]:
len(vocab)

324

In [13]:
df.apply(build_vocab, axis=1)

0     None
1     None
2     None
3     None
4     None
      ... 
85    None
86    None
87    None
88    None
89    None
Length: 90, dtype: object

In [18]:
# convert words to numerical indices
def text_to_indices(text, vocab):

  indexed_text = []

  for token in tokenize(text):

    if token in vocab:
      indexed_text.append(vocab[token])
    else:
      indexed_text.append(vocab['<UNK>'])

  return indexed_text

In [19]:
text_to_indices("What is that", vocab)

[1, 2, 0]

In [20]:
vocab

{'<UNK>': 0,
 'what': 1,
 'is': 2,
 'the': 3,
 'capital': 4,
 'of': 5,
 'france': 6,
 'paris': 7,
 'germany': 8,
 'berlin': 9,
 'who': 10,
 'wrote': 11,
 'to': 12,
 'kill': 13,
 'a': 14,
 'mockingbird': 15,
 'harper-lee': 16,
 'largest': 17,
 'planet': 18,
 'in': 19,
 'our': 20,
 'solar': 21,
 'system': 22,
 'jupiter': 23,
 'boiling': 24,
 'point': 25,
 'water': 26,
 'celsius': 27,
 '100': 28,
 'painted': 29,
 'mona': 30,
 'lisa': 31,
 'leonardo-da-vinci': 32,
 'square': 33,
 'root': 34,
 '64': 35,
 '8': 36,
 'chemical': 37,
 'symbol': 38,
 'for': 39,
 'gold': 40,
 'au': 41,
 'which': 42,
 'year': 43,
 'did': 44,
 'world': 45,
 'war': 46,
 'ii': 47,
 'end': 48,
 '1945': 49,
 'longest': 50,
 'river': 51,
 'nile': 52,
 'japan': 53,
 'tokyo': 54,
 'developed': 55,
 'theory': 56,
 'relativity': 57,
 'albert-einstein': 58,
 'freezing': 59,
 'fahrenheit': 60,
 '32': 61,
 'known': 62,
 'as': 63,
 'red': 64,
 'mars': 65,
 'author': 66,
 '1984': 67,
 'george-orwell': 68,
 'currency': 69,
 'unit

In [21]:
import torch
from torch.utils.data import Dataset, DataLoader

In [22]:
class QADataset(Dataset):

  def __init__(self, df, vocab):
    self.df = df
    self.vocab = vocab

  def __len__(self):
    return self.df.shape[0]

  def __getitem__(self, index):

    numerical_question = text_to_indices(self.df.iloc[index]['question'], self.vocab)
    numerical_answer = text_to_indices(self.df.iloc[index]['answer'], self.vocab)

    return torch.tensor(numerical_question), torch.tensor(numerical_answer)

In [23]:
dataset = QADataset(df, vocab)

In [24]:
dataloader = DataLoader(dataset, batch_size=1, shuffle=True)

In [25]:
for question, answer in dataloader:
  print(question, answer[0])

tensor([[10, 11, 12, 13, 14, 15]]) tensor([16])
tensor([[1, 2, 3, 4, 5, 8]]) tensor([9])
tensor([[ 10, 140,   3, 141, 270,  93, 271,   5,   3, 272]]) tensor([273])
tensor([[ 1,  2,  3,  4,  5, 73]]) tensor([74])
tensor([[ 42, 312,   2, 313,  62,  63,   3, 314, 315]]) tensor([316])
tensor([[  1,   2,   3, 180, 181, 182, 183]]) tensor([184])
tensor([[ 42, 255,   2, 256,  83, 257, 258]]) tensor([259])
tensor([[ 10,  75, 208]]) tensor([209])
tensor([[ 10,  11, 189, 158, 190]]) tensor([191])
tensor([[ 1,  2,  3, 33, 34,  5, 35]]) tensor([36])
tensor([[10, 75, 76]]) tensor([77])
tensor([[ 1,  2,  3, 37, 38, 39, 40]]) tensor([41])
tensor([[ 10,  11, 157, 158, 159]]) tensor([160])
tensor([[  1,   2,   3, 146, 147,  19, 148]]) tensor([149])
tensor([[  1,   2,   3,   4,   5, 236, 237]]) tensor([238])
tensor([[  1,   2,   3, 163, 164, 165,  83,  84]]) tensor([166])
tensor([[ 10,  75,   3, 296,  19, 297]]) tensor([298])
tensor([[ 78,  79, 129,  81,  19,   3,  21,  22]]) tensor([36])
tensor([[  1, 

In [29]:
import torch.nn as nn

In [58]:
class SimpleRNN(nn.Module):

  def __init__(self, vocab_size):
    super().__init__()
    self.embedding = nn.Embedding(vocab_size, embedding_dim=50)
    self.rnn = nn.RNN(50, 64, batch_first=True)
    self.fc = nn.Linear(64, vocab_size)

  def forward(self, question):
    embedded_question = self.embedding(question)
    hidden, final = self.rnn(embedded_question)
    output = self.fc(final.squeeze(0))

    return output

In [59]:
learning_rate=0.001
epochs=20


In [60]:
model=SimpleRNN(len(vocab))
criterion=nn.CrossEntropyLoss()
optimizer=torch.optim.Adam(model.parameters(),lr=learning_rate)

In [61]:
# training loop

for epoch in range(epochs):

  total_loss = 0

  for question, answer in dataloader:

    optimizer.zero_grad()

    # forward pass
    output = model(question)

    # loss -> output shape (1,324) - (1)
    loss = criterion(output, answer[0])

    # gradients
    loss.backward()

    # update
    optimizer.step()

    total_loss = total_loss + loss.item()

  print(f"Epoch: {epoch+1}, Loss: {total_loss:4f}")

Epoch: 1, Loss: 527.843358
Epoch: 2, Loss: 455.404415
Epoch: 3, Loss: 376.311614
Epoch: 4, Loss: 318.213651
Epoch: 5, Loss: 269.867080
Epoch: 6, Loss: 223.606730
Epoch: 7, Loss: 181.650769
Epoch: 8, Loss: 143.078843
Epoch: 9, Loss: 111.200369
Epoch: 10, Loss: 85.529015
Epoch: 11, Loss: 66.440732
Epoch: 12, Loss: 52.939914
Epoch: 13, Loss: 41.989667
Epoch: 14, Loss: 34.150864
Epoch: 15, Loss: 28.498804
Epoch: 16, Loss: 23.818320
Epoch: 17, Loss: 20.197560
Epoch: 18, Loss: 17.247818
Epoch: 19, Loss: 15.122159
Epoch: 20, Loss: 13.243582


In [76]:
def predict(model,question,threshold=0.5):

    numerical_question=text_to_indices(question,vocab)

    question_tensor=torch.tensor(numerical_question).unsqueeze(0)

    output=model(question_tensor)
    
    prob=torch.nn.functional.softmax(output),dim=1 

    value,index=torch.max(prob,dim=1)

    if(value<threshold):
        print("I don't know")
    
    print(list(vocab.keys()[index])

SyntaxError: invalid syntax. Maybe you meant '==' or ':=' instead of '='? (4210376143.py, line 9)

In [75]:
predict(model,"what is that")

torch.Size([1, 324])
